Demo of land cover regridding module.

In [1]:
import xarray as xr
import numpy as np
import random
import pandas as pd
from src import land_cover

random.seed(10)
np.random.seed(10)

Create a dummy land cover dataset to be regridded.

In [2]:
# dummy land cover dataset
dummy_data = np.random.randint(3, size=(1, 4, 5))
lat = np.arange(1, 3, 0.5)
lon = np.arange(12, 14.5, 0.5)
time = pd.Timestamp("2020-09-01")

In [3]:
ds = xr.Dataset(
    data_vars=dict(
        lccs_class=(["time", "lat", "lon"], dummy_data),
    ),
    coords=dict(
        time=[time],
        lat=(["lat"], lat),
        lon=(["lon"], lon),
    ),
    attrs=dict(description="Dummy land cover data."),
)
ds

<xarray.Dataset>
Dimensions:     (time: 1, lat: 4, lon: 5)
Coordinates:
  * time        (time) datetime64[ns] 2020-09-01
  * lat         (lat) float64 1.0 1.5 2.0 2.5
  * lon         (lon) float64 12.0 12.5 13.0 13.5 14.0
Data variables:
    lccs_class  (time, lat, lon) int64 1 1 0 0 1 0 1 1 0 1 1 2 0 1 0 2 0 2 0 0
Attributes:
    description:  Dummy land cover data.

Create a dataset with target grid.

In [4]:
# dummy target grid
dummy_data = np.random.randint(12, size=(3, 2, 2))
lat = np.array([1.2, 2.1])
lon = np.array([12.2, 12.9])
time = pd.date_range("2020-09-06", periods=3)

In [5]:
ds_target = xr.Dataset(
    data_vars=dict(
        temperature=(["time", "lat", "lon"], dummy_data),
    ),
    coords=dict(
        time=time,
        lat=(["lat"], lat),
        lon=(["lon"], lon),
    ),
    attrs=dict(description="Dummy target dataset."),
)
ds_target

<xarray.Dataset>
Dimensions:      (time: 3, lat: 2, lon: 2)
Coordinates:
  * time         (time) datetime64[ns] 2020-09-06 2020-09-07 2020-09-08
  * lat          (lat) float64 1.2 2.1
  * lon          (lon) float64 12.2 12.9
Data variables:
    temperature  (time, lat, lon) int64 4 11 6 8 11 10 1 8 4 1 3 6
Attributes:
    description:  Dummy target dataset.

Now we can simply call the regrid function from `land_cover` to perform most common class selection.

In [6]:
regrid_land_cover = land_cover.regrid(ds, ds_target)
regrid_land_cover

<xarray.Dataset>
Dimensions:     (time: 1, lat: 2, lon: 2)
Coordinates:
  * time        (time) datetime64[ns] 2020-09-01
  * lat         (lat) float64 1.2 2.1
  * lon         (lon) float64 12.2 12.9
Data variables:
    lccs_class  (time, lat, lon) int64 1 0 2 0
Attributes:
    description:  Dummy land cover data.

Now let's verify the regridding of land cover classes. <br>
According to our implementation in `land_cover`, we can get the resolution of target grid and calculate target regridding intervals, which are used to `groupby` the data points from land cover dataset, based on both the resolution and the target grid.

In [7]:
(cell_lat_target, cell_lon_target) = land_cover.infer_resolution(ds_target)

lat_bounds = land_cover._construct_intervals(ds_target["lat"].values, cell_lat_target)
lon_bounds = land_cover._construct_intervals(ds_target["lon"].values, cell_lon_target)
print("interval by latitudes", lat_bounds)
print("interval by longitudes", lon_bounds)

interval by latitudes IntervalIndex([[0.7499999999999999, 1.65), [1.65, 2.55)], dtype='interval[float64, left]')
interval by longitudes IntervalIndex([[11.849999999999998, 12.55), [12.55, 13.25)], dtype='interval[float64, left]')


Next, we can group the data points on land cover grid by the intervals of regridding. Here we only show the grid points corresponding to lat[0] and lon[0] (by index) on new grid.

In [8]:
sel_ds_index_0_0 = ds["lccs_class"].sel(
    lat=slice(lat_bounds[0].left, lat_bounds[0].right),
    lon=slice(lon_bounds[0].left, lon_bounds[0].right),
)
sel_ds_index_0_0.values

array([[[1, 1],
        [0, 1]]])

Now we can pick up the most common label for this point!

In [9]:
expected_most_common_index_0_0 = land_cover._most_common_label(sel_ds_index_0_0.values)
expected_most_common_index_0_0

1

Finally we can verify our regridding.

In [10]:
assert regrid_land_cover["lccs_class"].values[0,0,0] == expected_most_common_index_0_0

Now let's continue with a real land cover dataset, the Land cover classification gridded maps. <br>
(https://cds.climate.copernicus.eu/cdsapp#!/dataset/satellite-land-cover?tab=overview)

Let's load the dataset first. Note that this land cover data site has a spatial resolution of 300 meter, which makes it very large. Therefore let's drop the variables that are not needed.

In [11]:
from pathlib import Path

ds_land_cover = xr.open_dataset("/data/volume_2/land_cover/C3S-LC-L4-LCCS-Map-300m-P1Y-2016-v2.1.1.nc")
ds_land_cover = ds_land_cover.drop(list(ds_land_cover.keys())[1:])
ds_land_cover

<xarray.Dataset>
Dimensions:     (time: 1, lat: 64800, lon: 129600)
Coordinates:
  * lat         (lat) float64 90.0 90.0 89.99 89.99 ... -89.99 -90.0 -90.0
  * lon         (lon) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * time        (time) datetime64[ns] 2016-01-01
Data variables:
    lccs_class  (time, lat, lon) uint8 ...
Attributes: (12/38)
    id:                         C3S-LC-L4-LCCS-Map-300m-P1Y-2016-v2.1.1
    title:                      Land Cover Map of 2016
    summary:                    This dataset characterizes the land cover of ...
    type:                       C3S-LC-L4-LCCS-Map-300m-P1Y
    project:                    EC C3S Land cover
    references:                 https://cds.climate.copernicus.eu/
    ...                         ...
    geospatial_lon_max:         null
    spatial_resolution:         300m
    geospatial_lat_units:       degrees_north
    geospatial_lat_resolution:  0.002778
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  0.002778

For a demo purpose, we only make use a subset of data.

In [12]:
time_region_na = {
    "lat": slice(15, 40),
    "lon": slice(-140, -55),
}
ds_land_cover = ds_land_cover.sortby(["time", "lat", "lon"])
ds_land_cover = ds_land_cover.sel(time_region_na)
ds_land_cover

<xarray.Dataset>
Dimensions:     (time: 1, lat: 9000, lon: 30600)
Coordinates:
  * lat         (lat) float64 15.0 15.0 15.01 15.01 ... 39.99 39.99 40.0 40.0
  * lon         (lon) float64 -140.0 -140.0 -140.0 -140.0 ... -55.01 -55.0 -55.0
  * time        (time) datetime64[ns] 2016-01-01
Data variables:
    lccs_class  (time, lat, lon) uint8 ...
Attributes: (12/38)
    id:                         C3S-LC-L4-LCCS-Map-300m-P1Y-2016-v2.1.1
    title:                      Land Cover Map of 2016
    summary:                    This dataset characterizes the land cover of ...
    type:                       C3S-LC-L4-LCCS-Map-300m-P1Y
    project:                    EC C3S Land cover
    references:                 https://cds.climate.copernicus.eu/
    ...                         ...
    geospatial_lon_max:         null
    spatial_resolution:         300m
    geospatial_lat_units:       degrees_north
    geospatial_lat_resolution:  0.002778
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  0.002778

This data is too large to fit into the memory. Therefore we first make it coarse by a factor of 300 (with new resolution to be 9 km). The function will use `dask` to increase the processing speed and reduce the memory usage.

In [13]:
# let's call the client to grab the dask board
from distributed import Client
client = Client(n_workers=4, threads_per_worker=2)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 7.65 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36543,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 7.65 GiB
Comm: tcp://127.0.0.1:36867,Total threads: 2
Dashboard: http://127.0.0.1:39675/status,Memory: 1.91 GiB
Nanny: tcp://127.0.0.1:39173,


In [14]:
# call the coarsen
coarse_ds = land_cover.coarsen(ds_land_cover, 300, 300)

In [15]:
# start the calculation
coarse_ds = coarse_ds.compute()

In [16]:
coarse_ds

<xarray.Dataset>
Dimensions:     (time: 1, lat: 30, lon: 102)
Coordinates:
  * time        (time) datetime64[ns] 2016-01-01
  * lat         (lat) float64 15.42 16.25 17.08 17.92 ... 37.92 38.75 39.58
  * lon         (lon) float64 -139.6 -138.7 -137.9 ... -57.08 -56.25 -55.42
Data variables:
    lccs_class  (time, lat, lon) uint8 210 210 210 210 210 ... 210 210 210 210

Again, we create a new dataset containing the target grid.

In [17]:
# dummy target grid
dummy_data = np.random.randint(12, size=(3, 2, 2))
lat = np.array([17.2, 18.2])
lon = np.array([-59.2, -58.2])
time = pd.date_range("2020-09-06", periods=3)

ds_target = xr.Dataset(
    data_vars=dict(
        temperature=(["time", "lat", "lon"], dummy_data),
    ),
    coords=dict(
        time=time,
        lat=(["lat"], lat),
        lon=(["lon"], lon),
    ),
    attrs=dict(description="Dummy target dataset."),
)
ds_target

<xarray.Dataset>
Dimensions:      (time: 3, lat: 2, lon: 2)
Coordinates:
  * time         (time) datetime64[ns] 2020-09-06 2020-09-07 2020-09-08
  * lat          (lat) float64 17.2 18.2
  * lon          (lon) float64 -59.2 -58.2
Data variables:
    temperature  (time, lat, lon) int64 5 3 9 6 9 1 9 4 11 2 6 7
Attributes:
    description:  Dummy target dataset.

And finally, we can call the regridder!

In [19]:
regrid_land_cover = land_cover.regrid(coarse_ds, ds_target)
regrid_land_cover